In [ ]:
# Vibe Fashion Backend Test with Ollama Gemma 3B:12B on Google Cloud

This notebook tests the backend with Ollama using Gemma 3B:12B model on Google Cloud.

**Ollama Endpoint:** https://ollama-153939933605.europe-west1.run.app/
**Model:** gemma3:12b

In [ ]:
# Import required libraries
import requests
import json
import base64
import io
from PIL import Image
import time


In [ ]:
# Configuration
OLLAMA_BASE_URL = "https://ollama-153939933605.europe-west1.run.app"
BACKEND_URL = "http://localhost:8000"  # Local backend
MODEL_NAME = "gemma3:12b"

print(f"🔧 Configuration:")
print(f"   Ollama URL: {OLLAMA_BASE_URL}")
print(f"   Backend URL: {BACKEND_URL}")
print(f"   Model: {MODEL_NAME}")


In [ ]:
# Test 1: Check Ollama service connection
print("🔍 Testing Ollama connection...")

try:
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=10)
    if response.status_code == 200:
        models = response.json()
        print(f"✅ Ollama service is accessible")
        print(f"📋 Available models: {[model['name'] for model in models.get('models', [])]}")
        ollama_connected = True
    else:
        print(f"❌ Ollama service returned status {response.status_code}")
        ollama_connected = False
except requests.exceptions.RequestException as e:
    print(f"❌ Failed to connect to Ollama service: {e}")
    ollama_connected = False


In [ ]:
# Test 2: Direct Ollama chat test
if ollama_connected:
    print("\n🤖 Testing direct Ollama chat...")
    
    try:
        payload = {
            "model": MODEL_NAME,
            "messages": [
                {
                    "role": "user",
                    "content": "Hello! Can you help me with fashion advice? What should I wear for a casual dinner?"
                }
            ],
            "stream": False
        }
        
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/chat",
            json=payload,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Ollama chat successful")
            print(f"💬 Response: {result.get('message', {}).get('content', 'No content')}")
            ollama_chat_success = True
        else:
            print(f"❌ Ollama chat failed with status {response.status_code}")
            print(f"Response: {response.text}")
            ollama_chat_success = False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Ollama chat request failed: {e}")
        ollama_chat_success = False
else:
    print("⏭️  Skipping Ollama chat test - service not connected")
    ollama_chat_success = False


In [ ]:
# Test 3: Create test image for backend testing
def create_test_image():
    """Create a simple test image for testing"""
    # Create a simple fashion-related test image
    img = Image.new('RGB', (300, 400), color='lightblue')
    
    # Add some simple shapes to make it look like clothing
    from PIL import ImageDraw
    draw = ImageDraw.Draw(img)
    
    # Draw a simple shirt shape
    draw.rectangle([100, 50, 200, 200], fill='white', outline='black', width=2)
    draw.rectangle([80, 180, 220, 300], fill='navy', outline='black', width=2)
    
    # Convert to bytes
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    return img_bytes.getvalue()

# Create test image
test_image = create_test_image()
print(f"🖼️  Created test image: {len(test_image)} bytes")


In [ ]:
# Test 4: Test backend health (assuming backend is running)
print("\n🏥 Testing backend health...")

try:
    response = requests.get(f"{BACKEND_URL}/", timeout=10)
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Backend is healthy: {result}")
        backend_healthy = True
    else:
        print(f"❌ Backend health check failed with status {response.status_code}")
        backend_healthy = False
except requests.exceptions.RequestException as e:
    print(f"❌ Backend health check failed: {e}")
    print("💡 Make sure to start the backend server first:")
    print("   cd services/backend && python start_server.py")
    backend_healthy = False


In [ ]:
# Test 5: Test backend chat endpoint with image
if backend_healthy:
    print("\n💬 Testing backend chat endpoint...")
    
    try:
        # Prepare form data
        files = {
            'image': ('test_fashion.png', test_image, 'image/png')
        }
        data = {
            'text': 'What do you think about this outfit? Can you suggest some improvements?'
        }
        
        response = requests.post(
            f"{BACKEND_URL}/chat",
            files=files,
            data=data,
            timeout=60
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Backend chat successful")
            print(f"💬 Response text: {result.get('text', 'No text')}")
            print(f"🖼️  Images returned: {len(result.get('images', []))}")
            print(f"✅ Success: {result.get('success', False)}")
            backend_chat_success = True
        else:
            print(f"❌ Backend chat failed with status {response.status_code}")
            print(f"Response: {response.text}")
            backend_chat_success = False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Backend chat request failed: {e}")
        backend_chat_success = False
else:
    print("⏭️  Skipping backend chat test - backend not healthy")
    backend_chat_success = False


In [ ]:
# Test Summary
print("\n" + "=" * 60)
print("📊 TEST SUMMARY")
print("=" * 60)

tests = [
    ("Ollama Connection", ollama_connected),
    ("Ollama Chat", ollama_chat_success),
    ("Backend Health", backend_healthy),
    ("Backend Chat", backend_chat_success)
]

for test_name, result in tests:
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"{test_name}: {status}")

total_tests = len(tests)
passed_tests = sum([result for _, result in tests])

print(f"\n🎯 Overall: {passed_tests}/{total_tests} tests passed")

if passed_tests == total_tests:
    print("🎉 All tests passed! Backend is ready to use.")
elif passed_tests >= 2:
    print("⚠️  Most tests passed. Check failed tests above.")
else:
    print("❌ Multiple tests failed. Check configuration and setup.")
